In [30]:
import pandas as pd
from datasets import load_dataset
import string

data_train = load_dataset('csv', data_files= '/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Train_SL_L1.csv', header=None, names=['word', 'label'])

/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 13154 examples [00:00, 470244.23 examples/s]


In [32]:
data_train
#[{'text': '61-letni moški se je pojavil s kroničnim levim ušesom izcedekom, glavobolom na levi strani, diplopijo'},{'entity': 'levim ušesom izcedekom'}, {'entity':'glavobolom na levi strani'}, {'entity':'diplopijo'}]

DatasetDict({
    train: Dataset({
        features: ['word', 'label'],
        num_rows: 13154
    })
})

In [102]:
import string

char = '-'

if char in string.punctuation:
    print(f"{char} is punctuation")
else:
    print(f"{char} is not punctuation")

- is punctuation


In [105]:
a = ' csd' 
a += 'AAA_results_discussion.ipynb'
a

' csdAAA_results_discussion.ipynb'

In [137]:
class Slovenian_preprocessor():

    def __init__(self, data):
        self.data = data
    
    def preprocess(self):
        self.data['label'] = self.data['label'].astype(str)
        text = ''
        overall_entities = []
        entities = []
        current_entity = ''
        prev_word=''
        for _, row in self.data.iterrows():
            # print(f"word: {row['word']}, label: {row['label']}, prev_word: {prev_word}")
            if prev_word == '.' and len(text.split())>5:
                if current_entity:
                    entities.append({'entity': current_entity})
                    current_entity = ''
                # print(f"ENTRO NELLA COND")
                overall_entities.append({'text': text, 'entities': entities})
                text = ''
                entities = []
            word, label = row['word'], row['label']
            if text!='' or (word.strip() in string.punctuation) or (prev_word.strip() in string.punctuation):
                space = ' '
            else:
                space = ''
            text = text + space + word
            if label != 'O':
                if label.startswith('B-'):
                    if current_entity:
                        entities.append({'entity': current_entity})
                    current_entity = word
                elif label.startswith('I-') and current_entity:
                    current_entity += ' ' + word
            prev_word = word
        data_df = pd.DataFrame(columns=['text', 'entities'])
        for el in overall_entities:
            el['text'] = el['text'].strip() 
            data_df = pd.concat([data_df, pd.DataFrame([el])], ignore_index=True)
        self.data = data_df

In [138]:
preprocessor = Slovenian_preprocessor(data_train)
preprocessor.preprocess()
preprocessor.data

,text,entities
0,61 - letni moški se je pojavil s kroničnim lev...,"[{'entity': 'levim ušesom izcedekom'}, {'entit..."
1,MRI s kontrastom je pokazal asimetrične signal...,"[{'entity': 'signalne spremembe'}, {'entity': ..."
2,- 8 . živčnega kompleksa .,[]
3,pacient je bil zdravljen konzervativno z lokal...,"[{'entity': 'vrtoglavice'}, {'entity': 'diplop..."
4,"22 - letna ženska , brez prej diagnosticiranih...","[{'entity': 'dihalne stiske'}, {'entity': 'utr..."
...,...,...
663,dvaindvajsetletna ženska iz šrilanke z anamnez...,[{'entity': 'akutni vnetni demielinizirajoči p...
664,študija prevodnosti živca je razkrila varianto...,"[{'entity': 'AIDP'}, {'entity': 'guillain-barr..."
665,analiza likvorja po 2 tednih je bila normalna ...,[]
666,ona je bila zdravljena z intravenskim imunoglo...,[]


In [115]:
data = pd.read_csv('/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Train_SL_L1.csv', header=None, names=['word', 'label'])

data['label'] = data['label'].astype(str)
text = ''
overall_entities = []
entities = []
current_entity = ''

all_text = ''

# Iterate through the rows of the DataFrame
prev_word=''
next_word_is_period = False
for index, row in data.iterrows():
    all_text += row['word'] + ' ' 
    # print(f"word: {row['word']}, label: {row['label']}, prev_word: {prev_word}")
    if prev_word == '.' and len(text.split())>5:
        if current_entity:
            entities.append({'entity': current_entity})
            current_entity = ''
        # print(f"ENTRO NELLA COND")
        print({'text': text, 'entities': entities})
        overall_entities.append({'text': text, 'entities': entities})
        text = ''
        entities = []
    word, label = row['word'], row['label']
    if text!='' or (word.strip() in string.punctuation) or (prev_word.strip() in string.punctuation):
        print(f"word: {word}, prev_word: {prev_word}")
        space = ' '
    else:
        space = ''
    text = text + space + word
    if label != 'O':
        if label.startswith('B-'):
            if current_entity:
                entities.append({'entity': current_entity})
            current_entity = word
        elif label.startswith('I-') and current_entity:
            current_entity += ' ' + word
    prev_word = word

    

word: 61, prev_word: 
word: -, prev_word: 61
word: letni, prev_word: -
word: moški, prev_word: letni
word: se, prev_word: moški
word: je, prev_word: se
word: pojavil, prev_word: je
word: s, prev_word: pojavil
word: kroničnim, prev_word: s
word: levim, prev_word: kroničnim
word: ušesom, prev_word: levim
word: izcedekom, prev_word: ušesom
word: ,, prev_word: izcedekom
word: glavobolom, prev_word: ,
word: na, prev_word: glavobolom
word: levi, prev_word: na
word: strani, prev_word: levi
word: ,, prev_word: strani
word: diplopijo, prev_word: ,
word: povezano, prev_word: diplopijo
word: z, prev_word: povezano
word: vrtoglavico, prev_word: z
word: ,, prev_word: vrtoglavico
word: tinitusom, prev_word: ,
word: in, prev_word: tinitusom
word: slušno, prev_word: in
word: okvaro, prev_word: slušno
word: ., prev_word: okvaro
{'text': ' 61 - letni moški se je pojavil s kroničnim levim ušesom izcedekom , glavobolom na levi strani , diplopijo povezano z vrtoglavico , tinitusom in slušno okvaro .', 'ent

In [127]:
data_df = pd.DataFrame(columns=['text', 'entities'])
for el in overall_entities:
    el['text'] = el['text'].strip() 
    data_df = pd.concat([data_df, pd.DataFrame([el])], ignore_index=True)
data_df

,text,entities
0,61 - letni moški se je pojavil s kroničnim lev...,"[{'entity': 'levim ušesom izcedekom'}, {'entit..."
1,MRI s kontrastom je pokazal asimetrične signal...,"[{'entity': 'signalne spremembe'}, {'entity': ..."
2,- 8 . živčnega kompleksa .,[]
3,pacient je bil zdravljen konzervativno z lokal...,"[{'entity': 'vrtoglavice'}, {'entity': 'diplop..."
4,"22 - letna ženska , brez prej diagnosticiranih...","[{'entity': 'dihalne stiske'}, {'entity': 'utr..."
...,...,...
663,dvaindvajsetletna ženska iz šrilanke z anamnez...,[{'entity': 'akutni vnetni demielinizirajoči p...
664,študija prevodnosti živca je razkrila varianto...,"[{'entity': 'AIDP'}, {'entity': 'guillain-barr..."
665,analiza likvorja po 2 tednih je bila normalna ...,[]
666,ona je bila zdravljena z intravenskim imunoglo...,[]
